In [1]:
import pm4py
from pm4py.objects.log.obj import EventLog, Trace, Event
from demoparser2 import DemoParser
import pandas as pd
from pathlib import Path

In [2]:
parser=DemoParser("heroic-vs-3dmax-m1-dust2.dem")

In [28]:
b_dfusd = b_dfusd_evnts.iloc[i]
r_i = b_dfusd['total_rounds_played']
r_i

np.int32(7)

In [24]:
def entry_to_trace(i: int) -> list[str]:
    b_dfusd = b_dfusd_evnts.iloc[i]
    r_i = b_dfusd['total_rounds_played']
    end_t = b_dfusd['tick']

    b_plntd = b_plntd_evnts[b_plntd_evnts['total_rounds_played'] == r_i].iloc[0]
    start_t = b_plntd['tick']
    
    alive_dict = parser.parse_ticks(['is_alive'], ticks=[end_t])[['name', 'is_alive']].set_index('name')['is_alive'].to_dict()

    #trace = parser.parse_ticks(['last_place_name'], players=[player_id], ticks=range(start_t, end_t + 1))['last_place_name']
    # People who are dead when the bomb has been planted are included
    trace = parser.parse_ticks(['last_place_name', 'team_name', 'is_alive'], ticks=range(start_t, end_t + 1))
    team_mask = trace['team_name'] == 'CT'
    trace = trace[team_mask]
    first_tick = trace[trace['tick'] == start_t]
    alive_initial = first_tick[['name', 'is_alive']].set_index('name')['is_alive'].to_dict()
    logs = []
    
    for player in trace['name'].unique(): 
        if not alive_initial[player]:
            continue
        player_mask = trace['name'] == player 
        p_trace = trace[player_mask]
        #p_trace = p_trace['last_place_name']
        change_mask = p_trace['last_place_name'] != p_trace['last_place_name'].shift(1)
        p_trace = p_trace[change_mask]
        log = list(p_trace['last_place_name'])
        if not alive_dict[player]: 
            log.append('Die')
        log.append('Round End')
        logs.append(log)
    return logs

In [13]:
target_dir = Path('./Heroric_Dust2_demos') 
absolute_file_paths = [
    str(item.resolve())
    for item in target_dir.iterdir()
    if item.is_file() and 
    str(item.resolve()).endswith('.dem')
]

In [23]:
traces = []
for path in absolute_file_paths:
    print(f"Considering {path}")
    parser=DemoParser(path)
    
    parser_attributes = ['total_rounds_played']
    b_dfusd_evnts = parser.parse_event("bomb_defused", other=parser_attributes)
    b_plntd_evnts = parser.parse_event("bomb_planted", other=parser_attributes)

    count = len(b_dfusd_evnts)

    print(f"  Found {count} bomb defusions!")
    for i in range(count):
        trace = entry_to_trace(i)
        traces.extend(trace)
        print(f"  Parsed trace {i+1}")

Considering /home/kochc/projects/process-csgo-star-team/Heroric_Dust2_demos/lp-vs-kru-m1-dust2.dem
  Found 5 bomb defusions!
  Parsed trace 1


KeyboardInterrupt: 

In [22]:
event_log = EventLog()
event_log.attributes["concept:name"]= "csgo_demo_log"
for i, log in enumerate(fix):
    trace = Trace()
    trace.attributes["concept:name"] = f"{i}"
    for j, location in enumerate(log):
        event = Event()
        event['concept:name'] = location
        event['time:tick'] = j
        trace.append(event)
    event_log.append(trace)
pm4py.write_xes(event_log, "./logs/first.xes")